# Prediction Difference Computation Between Gender Group

Logic: 10 Folder Prediction on self-model vs average other groups

In [1]:
%pwd

'/mnt/d/OneDrive - Kyushu University/ESG09_Article/Code'

In [2]:
%cd ..

/mnt/d/OneDrive - Kyushu University/ESG09_Article


/home/linux/anaconda3/envs/automl/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package

In [3]:
from joblib import dump, load
import numpy as np
import os 
import pandas as pd
import random
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb

## Load dataset and divide them into subgroup

In [4]:
Df_Filename = os.path.join("Data", "GallupWB_Ml64var1911k14wave_v1.parquet")

In [5]:
Df = pd.read_parquet(Df_Filename)

In [6]:
Df.shape

(1911212, 64)

In [7]:
Df.columns

Index(['wave', 'INCOME_2', 'Cantril_ladder', 'Health_disable', 'Relative_have',
       'Living_standard_change', 'Enough_food', 'Enough_shelter',
       'Well_rested', 'Respected', 'Smile', 'Interesting_thing', 'Enjoyment',
       'Physical_pain', 'Worry', 'Sadness', 'Stress', 'Anger',
       'City_satisficied', 'Economic_change', 'Goodtime_job', 'Sat_pubtran',
       'Sat_road', 'Sat_edu', 'Sat_qualityair', 'Sat_qualitywater',
       'Sat_healthcare', 'Sat_affhouse', 'Sat_oppofriend', 'Good_minorities',
       'Good_gayles', 'Good_immigrants', 'Donated', 'Volunteer',
       'Help_stranger', 'Voice_official', 'Local_police', 'Safety_walk',
       'Stolen', 'Assualted', 'Religion_importance', 'Children_respected',
       'Children_learn', 'Women_respected', 'Sat_dealpoor', 'Sat_perserveenv',
       'Freedom_chooselife', 'Conf_military', 'Conf_judicial',
       'Conf_government', 'Conf_financial', 'Conf_honestyelections',
       'Freedom_media', 'Corruption_business', 'Corruption_governm

In [8]:
Df.sort_index().head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
367305,4,1.351047e+01,6.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,21.0,1.0,1.0,0.0,2.0,1.0,USA
367306,4,2.296780e+06,6.0,2.0,1.0,-0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,34.0,1.0,1.0,0.0,2.0,5.0,USA
367307,4,3.512722e+05,8.0,2.0,1.0,-1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,44.0,1.0,1.0,0.0,2.0,5.0,USA
367308,4,5.404188e+04,8.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,67.0,1.0,1.0,0.0,1.0,5.0,USA
367309,4,0.000000e+00,8.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,49.0,2.0,1.0,0.0,1.0,1.0,USA


In [9]:
Df = Df.sample(frac=1, random_state=42)

In [10]:
Df.head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1469152,10,35602.470662,7.0,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,64.0,1.0,5.0,0.0,2.0,3.0,NZL
1562723,11,20170.734058,5.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,18.0,1.0,6.0,0.0,2.0,1.0,LBN
708695,6,8664.458086,4.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,15.0,1.0,6.0,1.0,3.0,3.0,BWA
1828455,12,30706.925977,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,33.0,2.0,1.0,1.0,2.0,1.0,KWT
968859,7,4826.805232,8.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,29.0,8.0,1.0,1.0,3.0,4.0,HND


In [11]:
Df['COUNTRY_ISO3'] = Df['COUNTRY_ISO3'].astype('category')

In [12]:
Df.isna().any().any()

False

In [13]:
Df_male = Df.loc[Df['Gender_female']==0, :].drop(columns=['Gender_female'])

In [14]:
Df_female = Df.loc[Df['Gender_female']==1, :].drop(columns=['Gender_female'])

In [15]:
ym = Df_male['Cantril_ladder']
Xm = Df_male.drop(columns=['Cantril_ladder'])

In [16]:
yf = Df_female['Cantril_ladder']
Xf = Df_female.drop(columns=['Cantril_ladder'])

## Prediction of Each Group Based on Different Models

### Male Hyperparameter Reload, build model, and conduct 10 fold 10 times CV

In [17]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [18]:
bayes_search = load('Results/BayesSearchMale20iter.joblib')

In [19]:
print(f"Best parameters: {bayes_search.best_params_}")
print(f"Best score: {bayes_search.best_score_}")

Best parameters: OrderedDict([('gamma', 6.6360850776123375), ('learning_rate', 0.02519085390684862), ('max_delta_step', 3.0662510776734475), ('max_depth', 8), ('min_child_weight', 0.03343944487838535), ('n_estimators', 1136), ('reg_alpha', 0.001000562456331363), ('reg_lambda', 0.030390787275205566), ('subsample', 0.7971894567956928)])
Best score: 0.38039162083513095


In [20]:
dict(bayes_search.best_params_)

{'gamma': 6.6360850776123375,
 'learning_rate': 0.02519085390684862,
 'max_delta_step': 3.0662510776734475,
 'max_depth': 8,
 'min_child_weight': 0.03343944487838535,
 'n_estimators': 1136,
 'reg_alpha': 0.001000562456331363,
 'reg_lambda': 0.030390787275205566,
 'subsample': 0.7971894567956928}

In [21]:
CV_result = bayes_search.cv_results_

In [22]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).iloc[0,:]

mean_fit_time                                                      20.34373
std_fit_time                                                       0.268211
mean_score_time                                                    0.083779
std_score_time                                                     0.006058
param_gamma                                                        6.636085
param_learning_rate                                                0.025191
param_max_delta_step                                               3.066251
param_max_depth                                                           8
param_min_child_weight                                             0.033439
param_n_estimators                                                     1136
param_reg_alpha                                                    0.001001
param_reg_lambda                                                   0.030391
param_subsample                                                    0.797189
params      

In [21]:
male_xgb = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True, **bayes_search.best_params_)

In [22]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [23]:
scores = cross_val_score(male_xgb, Xm, ym, cv=kf, scoring='r2')

/home/linux/anaconda3/envs/automl/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:17:38] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [24]:
scores

array([0.37768344, 0.38304225, 0.37813157, 0.38565463, 0.38044917,
       0.37865251, 0.38220267, 0.37803065, 0.37701954, 0.38285246])

In [25]:
Xm.head()

,wave,INCOME_2,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,Smile,...,Corruption_business,Corruption_government,Performance_leadership,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1469152,10,35602.470662,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,64.0,1.0,5.0,0.0,2.0,3.0,NZL
708695,6,8664.458086,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,15.0,1.0,6.0,1.0,3.0,3.0,BWA
1828455,12,30706.925977,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,33.0,2.0,1.0,1.0,2.0,1.0,KWT
968859,7,4826.805232,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,29.0,8.0,1.0,1.0,3.0,4.0,HND
2375401,16,6999.233312,2.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,19.0,1.0,6.0,0.0,2.0,4.0,THA


In [26]:
ym.head()

1469152    7.0
708695     4.0
1828455    2.0
968859     8.0
2375401    5.0
Name: Cantril_ladder, dtype: float64

In [27]:
random.seed(42)

In [28]:
range_limit = 8192
num_samples = 10
sampled_integers = random.sample(range(range_limit), num_samples)
print(sampled_integers)

[1824, 409, 4506, 4012, 3657, 2286, 1679, 1424, 6912, 520]


In [29]:
X = Xm
y = ym
model = male_xgb

In [33]:
f_prediction_list = [np.array(yf.index), np.array(yf.to_list())]
fold_predictions_array_list = []
r2_scores_list = []
r2f_scores_list = []

for i, random_integer in enumerate(sampled_integers):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_integer)
    
    r2_scores = []
    r2f_scores = []
    fold_predictions = []
    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        this_folder = np.array([y_test.index, y_test, y_pred]).T
        fold_predictions.append(this_folder)
    
        # Female Predict
        yf_pred = model.predict(Xf)
        f_prediction_list.append(yf_pred)
        r2f = r2_score(yf, yf_pred)
        r2f_scores.append(r2f)
    
    print(f"Mean R2: {np.mean(r2_scores)}")
    print(f"Standard Deviation of R2: {np.std(r2_scores)}")
    print(f"R2 list: {r2_scores}")
    print(f"R2 Female list: {r2f_scores}")
    
    fold_predictions_array = pd.DataFrame(np.concatenate(fold_predictions, axis = 0), columns = ['index', 'Real_y', f'Predict_y_{i}'])
    fold_predictions_array_list.append(fold_predictions_array)
    r2_scores_list.append(r2_scores)
    r2f_scores_list.append(r2f_scores)


Mean R2: 0.38039025887864664
Standard Deviation of R2: 0.001657469545609444
R2 list: [0.3808871525685763, 0.3810656692690306, 0.3799796527022393, 0.3798214570197558, 0.3800799700836339, 0.38156993266736794, 0.3826225607561333, 0.382571818262035, 0.3778141873933467, 0.37749018806434687]
R2 Female list: [0.36208656675039286, 0.3617568366702755, 0.3620983335699067, 0.36178189800466887, 0.36168093814096836, 0.3620274829755653, 0.3619145491587964, 0.3618479826388422, 0.3616592521548462, 0.3618668593599341]
Mean R2: 0.38020591520250646
Standard Deviation of R2: 0.001387690146795807
R2 list: [0.37912939007241353, 0.38077460820422815, 0.3797413481420253, 0.3813808620806257, 0.3800186695876785, 0.3821397122869663, 0.3770062604700928, 0.37984630491616667, 0.38040266453473937, 0.38161933173012885]
R2 Female list: [0.36168884877699736, 0.3618623975778881, 0.3618094369791225, 0.3615700331662878, 0.36182246298516907, 0.36175508306538073, 0.36209864876263365, 0.3618688883140577, 0.36208597277103516, 

In [29]:
male_male = [0.3808871525685763, 0.3810656692690306, 0.3799796527022393, 0.3798214570197558, 0.3800799700836339, 
 0.38156993266736794, 0.3826225607561333, 0.382571818262035, 0.3778141873933467, 0.37749018806434687] + \
[0.37912939007241353, 0.38077460820422815, 0.3797413481420253, 0.3813808620806257, 0.3800186695876785, 
 0.3821397122869663, 0.3770062604700928, 0.37984630491616667, 0.38040266453473937, 0.38161933173012885] + \
[0.3806711833831917, 0.38051088615792006, 0.3817877600672457, 0.37684390534131373, 0.3788844242933378, 
 0.3827162795054775, 0.3807380037890531, 0.3804274610839332, 0.38008843674068826, 0.3803907062547509] + \
[0.3866590332823391, 0.3804047439441193, 0.37831899623383436, 0.37754479174731426, 0.3798960283148787, 
 0.3796767009087203, 0.38171559191234294, 0.38135270289287093, 0.3787138868497555, 0.38077809924898764] + \
[0.38126933007031305, 0.3811738666659563, 0.38429202534732787, 0.3774161103126996, 0.37633519140476046, 
 0.38111594199379695, 0.37890594201305194, 0.3780906520006573, 0.38435543658178284, 0.3794313291407089] + \
[0.38515593126795833, 0.3792945595671062, 0.38354875609094163, 0.38087795362485766, 0.3818379866885816, 
 0.37999377830864045, 0.37986010495646627, 0.3801968394359947, 0.3784128906348161, 0.3757642817791149] + \
[0.38017635433438446, 0.37871848130326646, 0.377013653009416, 0.37942390638046797, 0.37965730178464074, 
 0.38225579151250444, 0.38167594396499094, 0.38115406379697114, 0.3795473169853475, 0.38348651049197946] + \
[0.3827446544234495, 0.37910722195935687, 0.37774020597858027, 0.38495234514764787, 0.37852476239379196, 
 0.38230486090451243, 0.3768552054908285, 0.38261675618709, 0.38218993825986836, 0.3760189962866163] + \
[0.3785438605713263, 0.38137163590107803, 0.38321339078994887, 0.38096919642633553, 0.38332933550035253, 
 0.38365231838270575, 0.38251186662774395, 0.37878184764485934, 0.3780413303374458, 0.37284531267059984] + \
[0.37575926701703477, 0.3829847324283111, 0.3840351987266154, 0.3792358937893464, 0.38295808747160887, 
 0.38002300919781806, 0.38067232637499626, 0.3785563365876452, 0.3760278875727353, 0.3826002299456316]

print(np.mean(male_male))
print(np.max(male_male))
print(np.min(male_male))
print(np.std(male_male))

0.3803368760520898
0.3866590332823391
0.37284531267059984
0.002340972024036134


In [30]:
female_male = [0.36208656675039286, 0.3617568366702755, 0.3620983335699067, 0.36178189800466887, 0.36168093814096836,
0.3620274829755653, 0.3619145491587964, 0.3618479826388422, 0.3616592521548462, 0.3618668593599341] + \
[0.36168884877699736, 0.3618623975778881, 0.3618094369791225, 0.3615700331662878, 0.36182246298516907, 
 0.36175508306538073, 0.36209864876263365, 0.3618688883140577, 0.36208597277103516, 0.36193581622619164] + \
[0.3616854009612558, 0.3619954850013667, 0.36185192728167825, 0.36172255334297476, 0.36173875688129353, 
 0.3618605724036489, 0.36173686420765006, 0.36190980462321887, 0.3620099173518506, 0.3614793653899263] + \
[0.3617807038826317, 0.3616151965616574, 0.3621621799974799, 0.36169684939339064, 0.36169585086752065, 
 0.36160952843644556, 0.36188531615971886, 0.3616491309178419, 0.36196615107962926, 0.3620096151933865] + \
[0.3618384716816898, 0.3616663745353823, 0.3619179539381824, 0.36190455036939617, 0.3617690886157524, 
 0.36172614238768497, 0.3619631225984331, 0.3621421767590355, 0.3617863197958048, 0.36201572922076397] + \
[0.3618032960696481, 0.3620849926024353, 0.36178592295006107, 0.3616172255440546, 0.36192460373267044, 
 0.3618586268912475, 0.3617881052983394, 0.3616881898101135, 0.3617838028444108, 0.3618031850138762] + \
[0.36178505071710176, 0.36185055175480973, 0.36174209460892814, 0.36167438558571197, 0.3618187412315734, 
 0.36163265390962196, 0.3621225057891413, 0.3615174050682587, 0.3622812552876249, 0.3619914575177151] + \
[0.361719308552099, 0.36181674907137773, 0.3620577223057905, 0.36205721005408564, 0.36186703889341754, 
 0.36199428544670864, 0.36181880241193076, 0.3619380353136943, 0.3617550202158958, 0.36157430476742314] + \
 [0.36189230221394864, 0.3621485720633718, 0.3621427827219834, 0.3617165495768745, 0.3618543512720668, 
  0.3616706662540825, 0.36206538890105644, 0.36148884689117033, 0.3617361497590834, 0.3612635799500947] + \
[0.3615157388727134, 0.3620723339060725, 0.3617544022221927, 0.36188081866885713, 0.3618138303233075, 
 0.36186455633607184, 0.3618739570564071, 0.361897239525773, 0.36159068897890234, 0.36177087795271834]

print(np.mean(female_male))
print(np.max(female_male))
print(np.min(female_male))
print(np.std(female_male))

0.36183175550592167
0.3622812552876249
0.3612635799500947
0.0001761902908782245


### Male Model Female Prediction

Prediction is average of 10 fold * 10 times

In [34]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
female_preidiction_df = pd.DataFrame(np.array(f_prediction_list).T, columns = column_list)

In [35]:
female_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1562723.0,5.0,5.839088,5.834451,5.625898,5.621678,5.632700,5.804675,5.817439,5.879545,...,5.837131,5.769565,5.711668,5.747663,5.680957,5.770644,5.687633,5.750808,5.608838,5.637754
1,723699.0,6.0,6.264700,6.312768,6.338540,6.280410,6.408187,6.356091,6.227365,6.320326,...,6.284347,6.277956,6.286661,6.268738,6.251865,6.328773,6.332788,6.292277,6.498807,6.174890
2,1535061.0,2.0,4.122586,3.941179,4.076188,3.911150,3.818053,4.102979,4.018035,4.048829,...,3.998187,3.944230,4.047266,3.880664,3.942466,4.144531,3.996729,4.238452,3.866111,3.912003
3,1938251.0,2.0,3.959238,3.908072,3.837922,4.074171,3.833439,3.841711,3.826024,3.784009,...,3.769156,3.898711,3.765349,3.921235,3.912536,3.912592,3.797161,3.968856,3.917776,3.617374
4,1836183.0,10.0,7.166825,7.184285,7.199993,7.073222,7.159858,7.154918,7.073984,7.071355,...,7.253138,7.144835,7.110516,7.049429,7.036189,7.117004,7.126630,7.085889,7.184171,7.155372


In [36]:
female_preidiction_df['Predict_y'] = female_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [37]:
female_preidiction_df['Predict_y'].head()

0    5.744620
1    6.289086
2    4.028737
3    3.868628
4    7.140278
Name: Predict_y, dtype: float64

In [38]:
female_preidiction_df = female_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [39]:
female_preidiction_df['index'] = female_preidiction_df['index'].astype(int)

In [41]:
female_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1562723,5.0,5.744620
1,723699,6.0,6.289086
2,1535061,2.0,4.028737
3,1938251,2.0,3.868628
4,1836183,10.0,7.140278
5,1653464,3.0,6.280715
6,776224,7.0,5.653042
7,718598,5.0,5.405391
8,1701982,7.0,6.334415
9,1578430,7.0,6.298959


In [42]:
female_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbMaleModelFemalePrediction_v1.parquet'))

### Male Model Male Prediction

In [43]:
merged_male_prediction_df = None
for male_prediction_df in fold_predictions_array_list:
    male_prediction_df['index'] = male_prediction_df['index'].astype(int)
    male_prediction_df['Real_y'] = male_prediction_df['Real_y'].astype(int)
    print(male_prediction_df.columns)
    if merged_male_prediction_df is None:
        merged_male_prediction_df = male_prediction_df
    else:
        merged_male_prediction_df = pd.merge(merged_male_prediction_df, male_prediction_df, on=['index', 'Real_y'])

Index(['index', 'Real_y', 'Predict_y_0'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_1'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_2'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_3'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_4'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_5'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_6'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_7'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_8'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_9'], dtype='object')


In [44]:
merged_male_prediction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,Predict_y_8,Predict_y_9
0,1322632,7,4.413478,4.637781,4.429467,4.863088,4.325640,4.308519,4.614108,4.719594,4.554439,4.469615
1,2218620,5,6.596425,6.637515,6.606482,6.624826,6.622952,6.364815,6.236040,6.295045,6.254305,6.382856
2,563995,2,3.912592,3.867527,3.632151,3.707560,3.805271,3.602890,3.716228,3.797394,3.848176,3.717182
3,1467908,2,3.698674,3.663790,3.592609,3.798667,3.555281,3.564170,3.517401,3.534380,3.609425,3.673797
4,1205253,5,6.482972,6.496974,6.540684,6.458544,6.452550,6.456511,6.450431,6.474968,6.364278,6.504576


In [45]:
merged_male_prediction_df['Predict_y'] = merged_male_prediction_df.iloc[:,2:13].mean(axis = 1)

In [46]:
merged_male_prediction_df = merged_male_prediction_df[['index', 'Real_y', 'Predict_y']]

In [47]:
merged_male_prediction_df.head(10)

,index,Real_y,Predict_y
0,1322632,7,4.533573
1,2218620,5,6.462126
2,563995,2,3.760697
3,1467908,2,3.620819
4,1205253,5,6.468249
5,1084244,8,6.004445
6,2252652,3,5.050173
7,2530259,7,6.991234
8,1995045,2,5.035235
9,2554842,4,6.730033


In [48]:
merged_male_prediction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbMaleModelMalePrediction_v1.parquet'))

### Female Hyperparameter Reload, build model, and conduct 10 fold 10 times CV

In [23]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [24]:
bayes_search = load('Results/BayesSearchFemale20iter.joblib')

In [25]:
print(f"Best parameters: {bayes_search.best_params_}")
print(f"Best score: {bayes_search.best_score_}")

Best parameters: OrderedDict([('gamma', 0.0021168611234870433), ('learning_rate', 0.008018403760488199), ('max_delta_step', 0.9918343379663181), ('max_depth', 10), ('min_child_weight', 0.01633423890395618), ('n_estimators', 2553), ('reg_alpha', 0.001), ('reg_lambda', 0.3940525510663733), ('subsample', 0.6401413525274862)])
Best score: 0.37976389062078214


In [26]:
dict(bayes_search.best_params_)

{'gamma': 0.0021168611234870433,
 'learning_rate': 0.008018403760488199,
 'max_delta_step': 0.9918343379663181,
 'max_depth': 10,
 'min_child_weight': 0.01633423890395618,
 'n_estimators': 2553,
 'reg_alpha': 0.001,
 'reg_lambda': 0.3940525510663733,
 'subsample': 0.6401413525274862}

In [27]:
CV_result = bayes_search.cv_results_

In [28]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).iloc[0,:]

mean_fit_time                                                    107.072437
std_fit_time                                                       2.801465
mean_score_time                                                    0.224159
std_score_time                                                     0.009131
param_gamma                                                        0.002117
param_learning_rate                                                0.008018
param_max_delta_step                                               0.991834
param_max_depth                                                          10
param_min_child_weight                                             0.016334
param_n_estimators                                                     2553
param_reg_alpha                                                       0.001
param_reg_lambda                                                   0.394053
param_subsample                                                    0.640141
params      

In [53]:
female_xgb = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True, **bayes_search.best_params_)

In [54]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [55]:
scores = cross_val_score(female_xgb, Xf, yf, cv=kf, scoring='r2')

In [56]:
scores

array([0.37835638, 0.38297348, 0.37804209, 0.37746884, 0.37796181,
       0.38080827, 0.38146018, 0.38130876, 0.38042954, 0.3802625 ])

In [57]:
Xf.head()

,wave,INCOME_2,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,Smile,...,Corruption_business,Corruption_government,Performance_leadership,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1562723,11,20170.734058,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,18.0,1.0,6.0,0.0,2.0,1.0,LBN
723699,6,15287.346147,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,28.0,2.0,6.0,1.0,2.0,2.0,RUS
1535061,10,4467.534252,1.0,1.0,-0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,69.0,5.0,6.0,0.0,3.0,3.0,MNE
1938251,13,7202.172618,2.0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,47.0,3.0,1.0,0.0,3.0,2.0,BIH
1836183,12,52643.260274,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,62.0,1.0,6.0,0.0,2.0,5.0,MLT


In [58]:
yf.head()

1562723     5.0
723699      6.0
1535061     2.0
1938251     2.0
1836183    10.0
Name: Cantril_ladder, dtype: float64

In [59]:
random.seed(42)

In [60]:
range_limit = 8192
num_samples = 10
sampled_integers = random.sample(range(range_limit), num_samples)
print(sampled_integers)

[1824, 409, 4506, 4012, 3657, 2286, 1679, 1424, 6912, 520]


In [61]:
X = Xf
y = yf
model = female_xgb

In [62]:
m_prediction_list = [np.array(ym.index), np.array(ym.to_list())]
fold_predictions_array_list = []
r2_scores_list = []
r2m_scores_list = []

for i, random_integer in enumerate(sampled_integers):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_integer)
    
    r2_scores = []
    r2m_scores = []
    fold_predictions = []
    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        this_folder = np.array([y_test.index, y_test, y_pred]).T
        fold_predictions.append(this_folder)
    
        # Female Predict
        ym_pred = model.predict(Xm)
        m_prediction_list.append(ym_pred)
        r2m = r2_score(ym, ym_pred)
        r2m_scores.append(r2m)
    
    print(f"Mean R2: {np.mean(r2_scores)}")
    print(f"Standard Deviation of R2: {np.std(r2_scores)}")
    print(f"R2 list: {r2_scores}")
    print(f"R2 Male list: {r2m_scores}")
    
    fold_predictions_array = pd.DataFrame(np.concatenate(fold_predictions, axis = 0), columns = ['index', 'Real_y', f'Predict_y_{i}'])
    fold_predictions_array_list.append(fold_predictions_array)
    r2_scores_list.append(r2_scores)
    r2m_scores_list.append(r2m_scores)

Mean R2: 0.3799473640764341
Standard Deviation of R2: 0.0025191302058939517
R2 list: [0.378696228057846, 0.38233393964069007, 0.3806872615746483, 0.37508774277782664, 0.3825729992978738, 0.3774772538476733, 0.3825300607810711, 0.37723089694215417, 0.38197749425306926, 0.38087976359148856]
R2 Male list: [0.36622599129486244, 0.36628848110447443, 0.36638518085401994, 0.3664570646103327, 0.36655739550450683, 0.3663009441505646, 0.3663438144765392, 0.36644671634268744, 0.36635250432159105, 0.36631297562624565]
Mean R2: 0.379979422152383
Standard Deviation of R2: 0.0020835280029845448
R2 list: [0.3790075184781454, 0.3778806708999207, 0.38029264218278025, 0.3837069859766671, 0.3806184329717778, 0.3776797119914287, 0.3830095775775234, 0.37744562323697706, 0.37890374247562886, 0.3812493157329808]
R2 Male list: [0.36638911257614404, 0.3664396970968633, 0.36641103641589334, 0.3664830703570606, 0.366406640495918, 0.3663046553977187, 0.3665073517365369, 0.3661159053116505, 0.36644425521934487, 0.3

In [32]:
female_female = [0.378696228057846, 0.38233393964069007, 0.3806872615746483, 0.37508774277782664, 0.3825729992978738, 
0.3774772538476733, 0.3825300607810711, 0.37723089694215417, 0.38197749425306926, 0.38087976359148856] + \
[0.3790075184781454, 0.3778806708999207, 0.38029264218278025, 0.3837069859766671, 0.3806184329717778, 
 0.3776797119914287, 0.3830095775775234, 0.37744562323697706, 0.37890374247562886, 0.3812493157329808] + \
[0.38078350220282686, 0.3819176516122582, 0.3756415976587276, 0.3798406088601425, 0.38103355814465256, 
 0.3763403047363708, 0.38283715836966137, 0.3801557785098947, 0.37900947014684505, 0.38230233198650576] + \
[0.37792646509352934, 0.3800677807517411, 0.37969785613887563, 0.37897581290812044, 0.38265240771950515, 
 0.3821827257978566, 0.3815521861072618, 0.38125613909753353, 0.37558539140149827, 0.3808013188433649] + \
[0.3767537633128569, 0.38743099727441177, 0.3785326597791787, 0.38100241519199507, 0.37777930220780676, 
 0.37770936319229287, 0.38019792253887785, 0.3851232091899014, 0.3780507971850381, 0.3793016527157096] + \
[0.37902555588519826, 0.38250941550396467, 0.3837841060496543, 0.38252109598031403, 0.3779529691999566, 
 0.37726408401450195, 0.3794850558670395, 0.37616521183182927, 0.3855301883033846, 0.37645448265212256] + \
[0.3842498239364437, 0.3782404080009013, 0.3781326426613466, 0.3796890214013948, 0.3824074577813785, 
 0.3847041448835683, 0.37578641040796956, 0.37575747465547227, 0.3804680331422744, 0.3830614282581546] + \
[0.381812319440721, 0.3805604707424687, 0.3718008900176192, 0.37855006848764183, 0.38239162077664346, 
 0.3845695804571434, 0.38025463017523864, 0.38104548815503825, 0.38098780783678976, 0.3791846124928986] + \
[0.38022295105418824, 0.3845230432004194, 0.37807351981749693, 0.382346199176098, 0.3817719921801197, 
 0.3815235807672789, 0.37969262413035054, 0.37732165684618035, 0.3757156491134458, 0.3801099694172946] + \
[0.3663940929916465, 0.3663578316605298, 0.3662233595279606, 0.36648015394312927, 0.36666467806242764, 
 0.36629148533861333, 0.3663337374050354, 0.36638149334124914, 0.3665535207882925, 0.3663273002754842]

print(np.mean(female_female))
print(np.max(female_female))
print(np.min(female_female))
print(np.std(female_female))

0.3787136532899973
0.38743099727441177
0.3662233595279606
0.00485079289101012


In [33]:
male_female = [0.36622599129486244, 0.36628848110447443, 0.36638518085401994, 0.3664570646103327, 0.36655739550450683, 
0.3663009441505646, 0.3663438144765392, 0.36644671634268744, 0.36635250432159105, 0.36631297562624565] + \
[0.36638911257614404, 0.3664396970968633, 0.36641103641589334, 0.3664830703570606, 0.366406640495918, 
 0.3663046553977187, 0.3665073517365369, 0.3661159053116505, 0.36644425521934487, 0.3664689909220139] + \
[0.3664965230242322, 0.3664426754574539, 0.3664378147732146, 0.36639646962466177, 0.36620410985630836, 
 0.366227227625297, 0.36628026905897615, 0.36635515721629985, 0.36642952681331453, 0.36650179620801193] + \
[0.36636836548062324, 0.36625831735859327, 0.3663395408743808, 0.36619248472958266, 0.3667740165369415, 
 0.36678756725496264, 0.3663533425678762, 0.3665512351226323, 0.3662281904181802, 0.3661999246780453] + \
[0.3661941074608389, 0.3663966954638136, 0.36614335388918184, 0.36668041758940606, 0.3666466575064119, 
 0.36631554161793933, 0.3666765780711617, 0.3663952189286376, 0.3663703938069922, 0.36636636663521494] + \
[0.36617315666495387, 0.3664129030361314, 0.365991931701094, 0.36642428919124004, 0.36613191027216974, 
 0.36672726839457614, 0.3665307670598461, 0.36652206042033875, 0.36644993196619946, 0.3666083780756022] + \
[0.36625201514654215, 0.3663533399789891, 0.366645236368245, 0.36635934795034697, 0.3665816344682593, 
 0.3662754419830744, 0.3660965763890903, 0.3664017679058019, 0.36661199314764037, 0.36647481954697414] + \
[0.36641148711012983, 0.3663801218190508, 0.36648626495415904, 0.3662389027304823, 0.3663721022211589, 
 0.3665494758269626, 0.3665486633398609, 0.3661136467308461, 0.36654493777965536, 0.3663719527117002] + \
[0.3663614156235918, 0.36629033146546663, 0.366301546329855, 0.3663961653736387, 0.366522917908387, 
 0.36645739804960153, 0.3663666353015784, 0.3665325279307632, 0.366269135174085, 0.36672026272912817] + \
[0.3663940929916465, 0.3663578316605298, 0.3662233595279606, 0.36648015394312927, 0.36666467806242764, 
 0.36629148533861333, 0.3663337374050354, 0.36638149334124914, 0.3665535207882925, 0.3663273002754842]

print(np.mean(male_female))
print(np.max(male_female))
print(np.min(male_female))
print(np.std(male_female))

0.3663991998357575
0.36678756725496264
0.365991931701094
0.00015418680294683408


### Female Model Male Prediction

Prediction is average of 10 fold * 10 times

In [63]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
male_preidiction_df = pd.DataFrame(np.array(m_prediction_list).T, columns = column_list)

In [64]:
male_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1469152.0,7.0,7.179998,7.169359,7.110114,7.114609,7.046237,7.189165,7.081265,7.188010,...,7.342556,7.165565,7.147645,7.149748,7.109254,7.028602,7.153663,7.027450,7.160717,7.072621
1,708695.0,4.0,4.788220,4.461441,4.581176,4.540627,4.610817,4.548023,4.502110,4.815558,...,4.423770,4.565396,4.542029,4.609428,4.560171,4.624424,4.759196,4.260899,4.694597,4.446789
2,1828455.0,2.0,6.834692,6.879720,6.767813,6.852139,6.859030,6.849695,6.720863,6.801285,...,6.677359,6.887650,6.811440,6.848136,6.839103,6.890448,6.787124,6.788069,6.772766,6.670276
3,968859.0,8.0,5.305040,5.504973,5.278830,5.399392,5.255627,5.480446,5.287542,5.186207,...,5.370023,5.379074,5.408173,5.346770,5.279541,5.542669,5.242316,5.232035,5.322680,5.397290
4,2375401.0,5.0,6.071717,6.073183,5.969072,6.170063,6.166055,6.116441,5.950705,6.193364,...,6.023760,6.224649,6.107260,6.146920,6.130798,6.035076,5.914371,6.038098,6.096590,6.098917


In [65]:
male_preidiction_df['Predict_y'] = male_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [66]:
male_preidiction_df['Predict_y'].head()

0    7.138159
1    4.560510
2    6.806237
3    5.348697
4    6.071751
Name: Predict_y, dtype: float64

In [67]:
male_preidiction_df = male_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [68]:
male_preidiction_df['index'] = male_preidiction_df['index'].astype(int)

In [69]:
male_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1469152,7.0,7.138159
1,708695,4.0,4.560510
2,1828455,2.0,6.806237
3,968859,8.0,5.348697
4,2375401,5.0,6.071751
5,831522,5.0,3.228370
6,2069375,2.0,2.798753
7,1655669,10.0,3.930820
8,1398218,10.0,7.640960
9,1637923,2.0,3.380952


In [70]:
male_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbFemaleModelMalePrediction_v1.parquet'))

### Female Model Female Prediction

In [71]:
merged_female_prediction_df = None
for female_prediction_df in fold_predictions_array_list:
    female_prediction_df['index'] = female_prediction_df['index'].astype(int)
    female_prediction_df['Real_y'] = female_prediction_df['Real_y'].astype(int)
    print(female_prediction_df.columns)
    if merged_female_prediction_df is None:
        merged_female_prediction_df = female_prediction_df
    else:
        merged_female_prediction_df = pd.merge(merged_female_prediction_df, female_prediction_df, on=['index', 'Real_y'])

Index(['index', 'Real_y', 'Predict_y_0'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_1'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_2'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_3'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_4'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_5'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_6'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_7'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_8'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_9'], dtype='object')


In [72]:
merged_female_prediction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,Predict_y_8,Predict_y_9
0,520447,5,5.115666,5.229943,5.109724,5.143077,5.115220,5.045670,5.150040,5.232091,5.209785,4.994143
1,1398320,8,7.464622,7.504145,7.450959,7.420430,7.438122,7.415725,7.457869,7.451415,7.427966,7.431737
2,1767387,5,4.242178,4.154600,4.077366,4.340173,4.433428,4.073252,4.309819,4.120506,4.206805,4.236224
3,2097259,10,5.173368,4.957349,4.817544,5.005050,5.281789,5.240803,4.775723,4.932765,5.195347,5.000306
4,624137,3,3.238917,3.356781,3.365002,3.467746,3.325092,3.412952,3.405274,3.413189,3.385181,3.307482


In [73]:
merged_female_prediction_df['Predict_y'] = merged_female_prediction_df.iloc[:,2:13].mean(axis = 1)

In [74]:
merged_female_prediction_df = merged_female_prediction_df[['index', 'Real_y', 'Predict_y']]

In [75]:
merged_female_prediction_df.head(10)

,index,Real_y,Predict_y
0,520447,5,5.134536
1,1398320,8,7.446299
2,1767387,5,4.219435
3,2097259,10,5.038004
4,624137,3,3.367762
5,947435,8,7.391705
6,554737,7,7.326737
7,2279649,4,6.840872
8,2151240,6,7.347265
9,1172285,6,6.779715


In [76]:
merged_female_prediction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbFemaleModelFemalePrediction_v1.parquet'))